In [1]:
import matplotlib as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

# Modeling imports
from xgboost  import XGBClassifier
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import StratifiedKFold
import optuna
from optuna.samplers import TPESampler
import numpy as np

path = "/Users/tcaron/Documents/Python Scripts/KaggleS3E5/data/"

train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+"test.csv")
origin = pd.read_csv(path+"WineQT.csv")

In [2]:
def SelfSplitTrain(df,test_size=0.2):
    X = df.drop(columns=["Id","quality"])
    y = df[["quality"]].values
    X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=test_size,random_state=42)
    return (X_train, X_val, y_train, y_val)

In [3]:
def pca(train,test,columns=["pH","fixed acidity"]):
    target = "quality"
    df_trn = train.copy(deep = True)
    df_tst = test.copy(deep = True)
    df_trn[target] = df_trn[target].map({3:0,
                    4:1,
                    5:2,
                    6:3,
                    7:4,
                    8:5})
    pca_ = PCA(n_components=1 ,whiten= False)
    df_trn["pca_1"] = pca_.fit_transform(df_trn[columns])
    df_tst["pca_1"] = pca_.fit_transform(df_tst[columns])
    
    for cols in columns:
        for df in [df_trn,df_tst]:
            df.drop(cols, axis =1, inplace = True)
    return (df_trn,df_tst)

def sortie_prep(y_pred):
    return y_pred +3

In [4]:
Id = test[["Id"]]
train = pd.concat([train,origin],ignore_index=True)
train,test= pca(train, test)
X_test = test.drop(columns = "Id")
X_train,X_val,y_train,y_val = SelfSplitTrain(train)

def objective(trial):
    target_clases = train["quality"].value_counts()
    n_classes = target_clases.nunique()
    params_optuna = {
            'max_depth': trial.suggest_int('max_depth', 1, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'gamma': trial.suggest_float('gamma', 0.01, 1.0),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0001, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0001, 1.0),
            'objective' : " multi:softmax",
            'num_class': n_classes,
        }
    
    n=trial.suggest_int('n_cv', 3, 10)
    cv = StratifiedKFold(n,shuffle=True, random_state=42)
    fold_scores = []
    model = XGBClassifier(**params_optuna)
    model.fit(X_train,
              y_train,
              eval_set= [(X_val,y_val)],
              early_stopping_rounds = 50,
              verbose=500)

    pred_val = model.predict(X_val)

    score = cohen_kappa_score(y_val,pred_val, weights='quadratic')
    fold_scores.append(score)
    return np.mean(fold_scores)


study = optuna.create_study(direction='maximize', sampler = TPESampler())
study.optimize(func=objective, n_trials=500)
print(study.best_params)

In [5]:
params_xgb = {'max_depth': 9,
              'learning_rate': 0.10170219296989287, 
              'n_estimators': 939,
              'min_child_weight': 4,
              'gamma': 0.2873681880046375,
              'subsample': 0.6582602969527163,
              'colsample_bytree': 0.7214980206445512,
              'reg_alpha': 0.33596370591990027,
              'reg_lambda': 0.5061368887005409,
              'n_cv': 5}

def objective2(trial):
    target_clases = train["quality"].value_counts()
    n_classes = target_clases.nunique()
    params_optuna = {
            'lambda_l1': trial.suggest_float('lambda_l1', 0, 1),
             'lambda_l2': trial.suggest_float('lambda_l2', 1, 10),
             'num_leaves': trial.suggest_int('num_leaves', 40, 60),
             'feature_fraction': trial.suggest_float('feature_fraction', 0, 1),
             'bagging_fraction': trial.suggest_float('bagging_fraction', 0, 1),
             'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
             'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
             'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
             'max_depth': trial.suggest_int('max_depth', 1, 20),
             'num_iterations':trial.suggest_int('num_iterations', 100, 10000),
             'objective' : "multiclass",
             'metric' :'multi_logloss',
        }
    
    n=trial.suggest_int('n_cv', 3, 10)
    cv = StratifiedKFold(n,shuffle=True, random_state=42)
    fold_scores = []
    model = LGBMClassifier(**params_optuna)
    model.fit(X_train,
              y_train,
              eval_set= [(X_val,y_val)],
              early_stopping_rounds = 50,
              verbose=500)

    pred_val = model.predict(X_val)

    score = cohen_kappa_score(y_val,pred_val, weights='quadratic')
    fold_scores.append(score)
    return np.mean(fold_scores)


study = optuna.create_study(direction='maximize', sampler = TPESampler())
study.optimize(func=objective2, n_trials=500)
print(study.best_params)

In [7]:
params_lgbm= {'lambda_l1': 0.06032958694472297,
              'lambda_l2': 9.468684954887555,
              'num_leaves': 49,
              'feature_fraction': 0.42024961425671364,
              'bagging_fraction': 0.6392661414098254,
              'bagging_freq': 9,
              'min_child_samples': 43,
              'min_data_in_leaf': 14,
              'max_depth': 8,
              'num_iterations': 3723,
              'n_cv': 9}

In [8]:
train.drop(columns='Id',inplace=True)

In [9]:
test.drop(columns='Id',inplace=True)

In [18]:
features = list(X_train.columns)
TARGET = 'quality'
k=10
cv = StratifiedKFold(k, shuffle=True, random_state=42)
fold_scores = []
test_preds = []
oof_preds = []
oof_true = []
for i, (train_idx,val_idx) in enumerate(cv.split(train[features],
                                                 train[TARGET])):
    
    X_train = train.loc[train_idx, features]
    y_train = train.loc[train_idx, TARGET]
    X_val = train.loc[val_idx, features]
    y_val = train.loc[val_idx, TARGET]
    
    #**************XGB***************
    model1 = XGBClassifier(**params_xgb)
    model1.fit(X_train,
             y_train,
             eval_set= [(X_val,y_val)],
             early_stopping_rounds = 200,
             verbose=200)
    
    pred_val1 = model1.predict(X_val)
    
    score1 = cohen_kappa_score(y_val,pred_val1, weights='quadratic')
    #discard the predictions of poor performing models
    if score1>0.36:
        test_preds.append(model1.predict(test[features]))
        fold_scores.append(score1)
    
    #************** Light GBM **************
    model2 = LGBMClassifier(**params_lgbm)
    model2.fit(X_train,
             y_train,
             eval_set= [(X_val,y_val)],
             early_stopping_rounds = 200,
             verbose=200)
    
    pred_val2 = model2.predict(X_val)
    
    score2 = cohen_kappa_score(y_val,pred_val2, weights='quadratic')
    #discard the predictions of poor performing models
    if score2>0.36:
        test_preds.append(model2.predict(test[features]))
        fold_scores.append(score2)
    
    oof_preds.extend(np.mean([pred_val1,pred_val2],axis=0))
    oof_true.extend(y_val)
    print(f'=== Fold {i} Cohen Kappa Score {np.mean([score1,score2])} ===')

print(f'=== Average Cohen Kappa Score {np.mean(fold_scores)} ===')

[10:50:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "n_cv" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:50:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:1.70027


C:\Users\tcaron\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[200]	validation_0-mlogloss:1.11046
[253]	validation_0-mlogloss:1.14047


C:\Users\tcaron\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\tcaron\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\tcaron\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: n_cv
[LightGBM] [Warning] feature_fraction is set=0.42024961425671364, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42024961425671364
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=43 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] lambda_l1 is set=0.06032958694472297, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.06032958694472297
[LightGBM] [Warning] bagging_fraction is set=0.6392661414098254, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6392661414098254
[LightGBM] [Warning] lambda_l2 is set=9.468684954887555, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.468684954887555
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[200]	valid_0's multi_logloss: 1.07675
=== Fold 0 Cohen Kappa Score 0.4952705100646929 ===
[10:50:57] WARNING: C:/Users/Administrator/workspace/xgboost-win

...

[LightGBM] [Warning] Unknown parameter: n_cv
[LightGBM] [Warning] feature_fraction is set=0.42024961425671364, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42024961425671364
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=43 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] lambda_l1 is set=0.06032958694472297, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.06032958694472297
[LightGBM] [Warning] bagging_fraction is set=0.6392661414098254, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6392661414098254
[LightGBM] [Warning] lambda_l2 is set=9.468684954887555, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.468684954887555
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[200]	valid_0's multi_logloss: 1.04514
=== Fold 8 Cohen Kappa Score 0.45651221955788257 ===
[10:51:42] WARNING: C:/Users/Administrator/workspace/xgboost-wi

C:\Users\tcaron\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[200]	validation_0-mlogloss:1.03841
[261]	validation_0-mlogloss:1.05994


C:\Users\tcaron\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\tcaron\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\tcaron\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: n_cv
[LightGBM] [Warning] feature_fraction is set=0.42024961425671364, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42024961425671364
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=43 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] lambda_l1 is set=0.06032958694472297, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.06032958694472297
[LightGBM] [Warning] bagging_fraction is set=0.6392661414098254, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6392661414098254
[LightGBM] [Warning] lambda_l2 is set=9.468684954887555, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.468684954887555
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[200]	valid_0's multi_logloss: 0.974329
=== Fold 9 Cohen Kappa Score 0.5543320491352426 ===
=== Average Cohen Kappa Score 0.5207368806499122 ===


In [11]:
class OptunaRounder:

    def __init__(self, y_true, y_pred):
        self.y_true = y_true
        self.y_pred = y_pred
        self.labels = np.unique(y_true)

    def __call__(self, trial):
        thresholds = []
        for i in range(len(self.labels) - 1):
            low = max(thresholds) if i > 0 else min(self.labels)
            high = max(self.labels)
            t = trial.suggest_float(f't{i}', low, high)
            thresholds.append(t)
        try:
            opt_y_pred = self.adjust(self.y_pred, thresholds)
        except: return 0
        return cohen_kappa_score(self.y_true, opt_y_pred, weights='quadratic')

    def adjust(self, y_pred, thresholds):
        opt_y_pred = pd.cut(y_pred,
                            [-np.inf] + thresholds + [np.inf],
                            labels=self.labels)
        return opt_y_pred

In [14]:
optuna.logging.set_verbosity(optuna.logging.WARNING) 
objective = OptunaRounder(oof_true, oof_preds)
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, timeout=100)

C:\Users\tcaron\Anaconda3\lib\site-packages\optuna\samplers\_tpe\parzen_estimator.py:194: RuntimeWarning: invalid value encountered in subtract
  return np.log(np.exp(weighted_log_pdf - max_[:, np.newaxis]).sum(axis=1)) + max_
C:\Users\tcaron\Anaconda3\lib\site-packages\optuna\samplers\_tpe\parzen_estimator.py:194: RuntimeWarning: invalid value encountered in subtract
  return np.log(np.exp(weighted_log_pdf - max_[:, np.newaxis]).sum(axis=1)) + max_
C:\Users\tcaron\Anaconda3\lib\site-packages\optuna\samplers\_tpe\parzen_estimator.py:194: RuntimeWarning: invalid value encountered in subtract
  return np.log(np.exp(weighted_log_pdf - max_[:, np.newaxis]).sum(axis=1)) + max_
C:\Users\tcaron\Anaconda3\lib\site-packages\optuna\samplers\_tpe\parzen_estimator.py:194: RuntimeWarning: invalid value encountered in subtract
  return np.log(np.exp(weighted_log_pdf - max_[:, np.newaxis]).sum(axis=1)) + max_
C:\Users\tcaron\Anaconda3\lib\site-packages\optuna\samplers\_tpe\parzen_estimator.py:194: Ru

In [21]:
best_thresholds = sorted(study.best_params.values())
print(f'Optimized thresholds: {best_thresholds}')
test_preds = np.array(test_preds).mean(axis=0) 
opt_test_preds = objective.adjust(test_preds, best_thresholds).astype(int) +3
Id["quality"]=opt_test_preds
print(Id.head(3))

Optimized thresholds: [0.4652585475560227, 1.6794102627797107, 2.502690035308638, 3.4846453830703283, 4.194506308210782]
     Id  quality
0  2056        6
1  2057        5
2  2058        6


In [23]:
Id['quality'].value_counts()

6    630
5    600
7    142
Name: quality, dtype: int64

In [24]:
Id.to_csv("sample_submission.csv",index=False)